In [7]:
# Функция для сортировки операций по убыванию длительности операции с учетом веса
def sort_operations_by_duration_and_weight(operations, operation_times, weights):
    sorted_operations = sorted(operations, key=lambda op: (operation_times[op], sum(1 if weight == 1 else weight for weight in weights[op].values())), reverse=True)
    return sorted_operations

# Функция для назначения операций в отделения с учетом дней и весов
def assign_operations_to_rooms(sorted_operations, rooms, days, operation_times, anesthesia_times, room_daily_times, weights):
    assigned_operations = {room: {day: [] for day in days} for room in rooms}
    waiting_operations = []

    for operation in sorted_operations:
        operation_time = operation_times[operation] + anesthesia_times[operation]
        assigned = False

        # Попытка назначить операцию в свое отделение (weight = 1)
        for room in rooms:
            for day in days:
                if weights[operation][room] == 1 and room_daily_times[room][day] >= operation_time:
                    assigned_operations[room][day].append(operation)
                    room_daily_times[room][day] -= operation_time
                    assigned = True
                    break
            if assigned:
                break

        # Если операцию нельзя назначить в свое отделение, попробовать назначить в другие (по возрастанию weights)
        if not assigned:
            sorted_rooms = sorted(rooms, key=lambda room: weights[operation][room])
            for room in sorted_rooms:
                for day in days:
                    if room_daily_times[room][day] >= operation_time:
                        assigned_operations[room][day].append(operation)
                        room_daily_times[room][day] -= operation_time
                        assigned = True
                        break
                if assigned:
                    break

        # Если операцию нельзя назначить, добавить ее в список ожидания
        if not assigned:
            waiting_operations.append(operation)

    # Попытка распределить операции из списка ожидания
    while waiting_operations:
        min_operation_time = min(operation_times[op] + anesthesia_times[op] for op in waiting_operations)
        # Сортируем операционные по свободной вместимости
        sorted_rooms = sorted(rooms, key=lambda r: min(room_daily_times[r].values()))

        for room in sorted_rooms:
            # Ищем день с наибольшей свободной вместимостью
            max_free_time = max(room_daily_times[room].values())
            for day in days:
                if room_daily_times[room][day] <= max_free_time:
                    # Пытаемся назначить операцию в эту операционную и день
                    for operation in waiting_operations[:]:
                        operation_time = operation_times[operation] + anesthesia_times[operation]
                        if room_daily_times[room][day] >= operation_time:
                            assigned_operations[room][day].append(operation)
                            waiting_operations.remove(operation)
                            room_daily_times[room][day] -= operation_time
                            if not waiting_operations:
                                break
                    # Если больше нет операций в ожидании, выходим из цикла по операционным
                    if not waiting_operations:
                        break

    return assigned_operations, waiting_operations

import pandas as pd

# Считываем данные из первой таблицы
data = pd.read_excel('/content/test data big.xlsx')

# Создаем множества уникальных операционных, отделений и дней
operations = data['Operation ID'].unique().tolist()
rooms = data['Operating Room'].unique().tolist()
departments = data['Department'].unique().tolist()
days = range(1, 10)

# Создаем словари для времени операций и анестезии
operation_times = dict(zip(operations, data['Operation Time']))
anesthesia_times = dict(zip(operations, data['Anesthesia']))

# Считываем данные из второй таблицы (матрица весов)
weights_data = pd.read_excel('/content/test matrix big.xlsx', index_col=0, header=0)
# Создаем словарь весов
weights = {}
for operation in operations:
    weights[operation] = {}
    for room in rooms:
        weights[operation][room] = weights_data.loc[operation, room]

# Создаем словарь для времени операционных
room_daily_times = {room: {day: 720 for day in days} for room in rooms}

sorted_operations = sort_operations_by_duration_and_weight(operations, operation_times, weights)
assigned_operations, waiting_operations = assign_operations_to_rooms(sorted_operations, rooms, days, operation_times, anesthesia_times, room_daily_times, weights)

print("Назначенные операции:")
for day in days:
    for room in rooms:
        if assigned_operations[room][day]:
            print(f"День {day}, операционная {room}: {assigned_operations[room][day]}")

print("\nОперации в ожидании:")
print(waiting_operations)


Назначенные операции:
День 1, операционная 2_9: [1, 192]
День 1, операционная 4_6: [260, 40]
День 1, операционная 1_9: [3, 30, 11]
День 1, операционная 6_5: [257, 38]
День 1, операционная 8_5: [22, 141, 277, 275]
День 1, операционная 5_3: [128, 296]
День 1, операционная 9_4: [127, 297]
День 1, операционная 8_2: [259, 41]
День 1, операционная R3_4: [20, 21, 54, 82, 199]
День 1, операционная R3_8: [131, 188, 200]
День 1, операционная 3_1: [198, 34]
День 1, операционная 10_10: [263, 184]
День 1, операционная R3_1: [266, 60, 212, 101]
День 1, операционная R1_6: [201, 133, 75]
День 1, операционная R1_10: [208, 242, 240]
День 2, операционная 2_9: [298, 162]
День 2, операционная 4_6: [226, 71, 228]
День 2, операционная 1_9: [112, 250, 90]
День 2, операционная 6_5: [98, 64, 47]
День 2, операционная 8_5: [7, 80, 6, 170]
День 2, операционная 5_3: [258, 14]
День 2, операционная 9_4: [67, 68, 81]
День 2, операционная 8_2: [159, 196, 16]
День 2, операционная R3_4: [104, 129, 139, 237, 231]
День 2, 